<a href="https://colab.research.google.com/github/LCaravaggio/DeepLearning_ITBA/blob/main/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c cifar100-dl-itba-2020q2

In [5]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [6]:
import numpy as np

x_test=np.load("x_test.npy")
x_train=np.load("x_train.npy")
y_train_coarse=np.load("y_train_coarse.npy")
y_train_fine=np.load("y_train_fine.npy")

#for file in os.listdir():
#    if file.endswith('.npy'):
#      np.load(file)

In [ ]:
if not os.path.exists('mlp_helper.py'):
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/mlp_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/fnn_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/MNIST_helper.py
     !wget https://github.com/lab-ml-itba/MLP-2019/raw/master/draw_nn.py

In [8]:
import os.path
import tensorflow.keras
import numpy as np
from matplotlib import pyplot as plt
from fnn_helper import PlotLosses
from MNIST_helper import plot_numbers, visualize_input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from fnn_helper import PlotLosses
#from keras_contrib.callbacks import CyclicLR
from matplotlib import pyplot as plt
from mlp_helper import plot_confusion_matrix
#import draw_nn

In [ ]:
np.unique(y_train_fine)

In [17]:
def get_model(activations=['tanh', 'sigmoid']):
    output_size = 100
    model_plane = Sequential()
    model_plane.add(Flatten(input_shape=x_train.shape[1:]))
    model_plane.add(Dense(10, activation=activations[0],  name='middle1', kernel_initializer='normal'))
    model_plane.add(Dense(5, activation=activations[1],  name='middle2', kernel_initializer='normal'))
    #model_plane.add(LeakyReLU())
    #model_plane.add(LeakyReLU())
    model_plane.add(Dense(output_size, activation='linear',  kernel_initializer='normal', name='salida'))
    return model_plane

In [ ]:
model_plane = get_model(['tanh', 'sigmoid'])
model_plane.summary()

In [35]:
batch_size=10

In [36]:
model_plane = get_model(activations=['tanh', 'linear'])

checkpointer = ModelCheckpoint(filepath='two-layer-2d.mnist.hdf5', verbose=1, save_best_only=True, mode='max', monitor='val_accuracy')
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=None, x_val=x_train, y_val_categorical=y_train_fine)
# clr = CyclicLR(1e-6, 1e-4, 8*int(len(x_train)/batch_size))

In [37]:
optim = RMSprop(lr=1e-4) # 1e-06
model_plane.compile( loss= 'mse', optimizer=optim, metrics=['accuracy'])

In [ ]:
model_plane.fit(x_train, 
          y_train_fine ,
          epochs=10, batch_size=batch_size, 
          #verbose=1, 
          validation_data=(x_train, y_train_fine), 
          callbacks=[checkpointer],
         )

In [ ]:
model_plane.evaluate(x_train, y_train_fine)